In [50]:
from qiskit import IBMQ, Aer
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit,QuantumRegister, execute, ClassicalRegister, transpile
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.circuit.library import MCXGate
import qiskit.quantum_info as qi
import math
from math import sqrt

In [51]:
def get_beta(s, j, alpha):
    numerator = 0;
    denominator = 0;
    l = 1
    while(l <= 2**(s-1)):
        idx = ((2*j) -1)* 2**(s-1) + l;
        numerator = numerator + (alpha[idx-1]*alpha[idx-1]);
        l = l+1
    l = 1
    while(l <= 2**(s)):
        idx = (j-1)* 2**s + l
        l = l+1
        denominator = denominator + (alpha[idx-1]*alpha[idx-1]);
    if numerator == 0 or denominator == 0:
        return 0;
    theta = 2 * math.asin(sqrt(numerator/denominator))
    print(s,j,theta)
    return theta

In [52]:
def get_rgate(angle, nbr_ctrl):
    qc = QuantumCircuit(1)
    qc.ry(angle,0)
    if(nbr_ctrl != 0):
        qc = qc.to_gate().control(nbr_ctrl)
    else:
        qc = qc.to_gate()
    return qc

In [59]:
state = [sqrt(0.1), 0, sqrt(0.9), 0]

In [60]:
qc = QuantumCircuit(2,2)
rgate1_1 = get_rgate(get_beta(1,1,state),1)
rgate1_2 = get_rgate(get_beta(1,2,state),1)
rgate2_1 = get_rgate(get_beta(2,1,state),0)

qc.barrier()
qc.append(rgate2_1, [1])
qc.barrier()
#For 0 state of q_1
qc.x(1)
qc.append(rgate1_1, [1,0])
qc.x(1)
qc.barrier()
#For 1 state of q_1
qc.append(rgate1_2, [1,0])
qc.barrier()
qc.measure(range(2), range(2))
qc.draw()

2 1 2.498091544796509


░                 ░      ┌─────────────┐      ░ ┌─────────────┐ ░ ┌─┐   
q_0: ─░─────────────────░──────┤ circuit-450 ├──────░─┤ circuit-458 ├─░─┤M├───
      ░ ┌─────────────┐ ░ ┌───┐└──────┬──────┘┌───┐ ░ └──────┬──────┘ ░ └╥┘┌─┐
q_1: ─░─┤ circuit-466 ├─░─┤ X ├───────■───────┤ X ├─░────────■────────░──╫─┤M├
      ░ └─────────────┘ ░ └───┘               └───┘ ░                 ░  ║ └╥┘
c: 2/════════════════════════════════════════════════════════════════════╩══╩═
                                                                         0  1

In [61]:

backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend)
result = job.result()
counts = job.result().get_counts()
print(counts)

{'10': 915, '00': 109}
